In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm

D:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Anaconda\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
dataset = pd.read_csv('data/HT_Sensor_dataset.dat',sep = '  ',skiprows=1,header = None,engine='python')
dataset.columns = ['id','time','R1','R2','R3','R4','R5','R6','R7','R8','Temp.','Humidity']
dataset.set_index('id',inplace = True)
dataset.head()

,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity
id,,,,,,,,,,,
0,-0.999750,12.8621,10.3683,10.4383,11.6699,13.4931,13.3423,8.04169,8.73901,26.2257,59.0528
0,-0.999472,12.8617,10.3682,10.4375,11.6697,13.4927,13.3412,8.04133,8.73908,26.2308,59.0299
0,-0.999194,12.8607,10.3686,10.4370,11.6696,13.4924,13.3405,8.04101,8.73915,26.2365,59.0093
0,-0.998916,12.8602,10.3686,10.4370,11.6697,13.4921,13.3398,8.04086,8.73936,26.2416,58.9905
0,-0.998627,12.8595,10.3688,10.4374,11.6699,13.4919,13.3390,8.04087,8.73986,26.2462,58.9736


In [4]:
output = pd.read_csv('data/HT_Sensor_metadata.dat',skiprows=1,sep = '\t',header = None)
output.columns = ['id','date','class','t0','dt']
output.head()

,id,date,class,t0,dt
0,0,07-04-15,banana,13.49,1.64
1,1,07-05-15,wine,19.61,0.54
2,2,07-06-15,wine,19.99,0.66
3,3,07-09-15,banana,6.49,0.72
4,4,07-09-15,wine,20.07,0.53


# 将元数据和主数据连接起来

In [5]:
dataset = dataset.join(output,how = 'inner')
dataset.set_index(np.arange(dataset.shape[0]),inplace = True)
dataset['time']  += dataset['t0']
dataset.drop(['t0'],axis = 1,inplace=True)
dataset.head()

,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp.,Humidity,id,date,class,dt
0,12.490250,12.8621,10.3683,10.4383,11.6699,13.4931,13.3423,8.04169,8.73901,26.2257,59.0528,0,07-04-15,banana,1.64
1,12.490528,12.8617,10.3682,10.4375,11.6697,13.4927,13.3412,8.04133,8.73908,26.2308,59.0299,0,07-04-15,banana,1.64
2,12.490806,12.8607,10.3686,10.4370,11.6696,13.4924,13.3405,8.04101,8.73915,26.2365,59.0093,0,07-04-15,banana,1.64
3,12.491084,12.8602,10.3686,10.4370,11.6697,13.4921,13.3398,8.04086,8.73936,26.2416,58.9905,0,07-04-15,banana,1.64
4,12.491373,12.8595,10.3688,10.4374,11.6699,13.4919,13.3390,8.04087,8.73986,26.2462,58.9736,0,07-04-15,banana,1.64


In [3]:
# 对训练集及测试集进行处理，将标签数字化

In [10]:
xtrain,xtest,ytrain,ytest = train_test_split(dataset[[u'R1',u'R2',u'R3',u'R4',u'R5',u'R6',u'R7',u'R8',u'Temp.',u'Humidity']].values,dataset['class'].values,train_size = 0.7)
for i in range(ytrain.shape[0]):
    if(ytrain[i] == 'background'):
        ytrain[i] = 0
    elif(ytrain[i] == 'banana'):
        ytrain[i] = 1
    else:
        ytrain[i] = 2
        
for i in range(ytest.shape[0]):
    if(ytest[i] == 'background'):
        ytest[i] = 0
    elif(ytest[i] == 'banana'):
        ytest[i] = 1
    else:
        ytest[i] = 2
        
ytrain = ytrain.astype('int64')
ytest = ytest.astype('int64')

In [11]:
xtrain_dataframe = pd.DataFrame(xtrain)
ytrain_dataframe = pd.DataFrame(ytrain)
xtest_dataframe = pd.DataFrame(xtest)
ytest_dataframe = pd.DataFrame(ytest)
xtrain_dataframe.columns = [u'R1',u'R2',u'R3',u'R4',u'R5',u'R6',u'R7',u'R8',u'Temp.',u'Humidity']
ytrain_dataframe.columns = ['class']
xtest_dataframe.columns = [u'R1',u'R2',u'R3',u'R4',u'R5',u'R6',u'R7',u'R8',u'Temp.',u'Humidity']
ytest_dataframe.columns = ['class']
res = sm.RLM(ytrain_dataframe, xtrain_dataframe).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:                  class   No. Observations:               650293
Model:                            RLM   Df Residuals:                   650283
Method:                          IRLS   Df Model:                            9
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Sun, 10 Mar 2019                                         
Time:                        00:37:30                                         
No. Iterations:                    46                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
R1             0.4019      0.002    229.853      0.000       0.398       0.405
R2            -0.1685      0.003    -64.711      0.000      -0.174      -0.163
R3            -0.2503      0.002   -157.863      0.000      -0.253      -0.247
R4             0.0940      0.003     33.452      0.000       0.089       0.100
R5             0.0048   5.36e-05     90.143      0.000       0.005       0.005
R6             0.0129      0.000     44.377      0.000       0.012       0.013
R7            -0.0233      0.001    -46.116      0.000      -0.024      -0.022
R8            -0.0216      0.000    -47.531      0.000      -0.023      -0.021
Temp.         -0.0710      0.001   -104.892      0.000      -0.072      -0.070
Humidity       0.0153      0.000     77.927      0.000       0.015       0.016
==============================================================================

If the model instance has been used for another fit with different fit
parameters, then the fit options might not be the correct ones anymore .
"""

In [12]:
xtrain,xtest,ytrain,ytest = train_test_split(dataset[[u'R2',u'R3',u'R4',u'R5',u'R6',u'R7',u'R8',u'Temp.',u'Humidity']].values,dataset['class'].values,train_size = 0.7)

In [13]:
for i in range(ytrain.shape[0]):
    if(ytrain[i] == 'background'):
        ytrain[i] = 0
    elif(ytrain[i] == 'banana'):
        ytrain[i] = 1
    else:
        ytrain[i] = 2
        
for i in range(ytest.shape[0]):
    if(ytest[i] == 'background'):
        ytest[i] = 0
    elif(ytest[i] == 'banana'):
        ytest[i] = 1
    else:
        ytest[i] = 2
        
ytrain = ytrain.astype('int64')
ytest = ytest.astype('int64')

# Support Vector Machine

In [4]:
from sklearn.svm import SVC

C_2d_range = [1e-2]
gamma_2d_range = [1e-1]
classifiers = []
for C in C_2d_range:
    for gamma in gamma_2d_range:
        clf = SVC(C=C, gamma=gamma)
        clf.fit(xtrain, ytrain)
        classifiers.append((C, gamma, clf))

NameError: name 'xtrain' is not defined

In [15]:
accuracy_score(clf.predict(xtest),ytest)

0.9337239592677379

In [16]:
accuracy_score(clf.predict(xtrain),ytrain)

0.9339251691160754

In [19]:
xtest[:10,:]

array([[ 7.19541,  7.31552,  8.20657, 10.4465 , 11.3277 ,  4.75292,
         4.63383, 26.3409 , 72.9365 ],
       [ 4.16483,  2.75635,  5.50571,  7.73716, 13.8143 ,  4.11486,
         5.54962, 28.8068 , 52.0205 ],
       [ 5.62683,  4.12958,  6.20219, 30.3771 , 10.8113 ,  2.09492,
         2.09478, 27.6076 , 63.5693 ],
       [ 6.65735,  4.5342 ,  8.37782,  5.12363, 11.1531 ,  2.16598,
         2.31212, 27.7533 , 58.7369 ],
       [ 5.09778,  5.25595,  5.93128, 11.2385 , 15.9221 ,  7.11002,
         4.77202, 28.008  , 62.9164 ],
       [10.4747 , 10.5822 , 11.8082 ,  5.67548, 28.5176 ,  6.85122,
         7.15645, 26.8546 , 57.1474 ],
       [10.3693 , 10.5153 , 11.6853 , 11.1131 , 17.4128 ,  6.23007,
         6.69596, 28.0649 , 52.4689 ],
       [10.0896 , 10.1999 , 11.4003 , 10.9111 , 15.4477 ,  6.89694,
         7.33591, 27.8848 , 50.3989 ],
       [ 7.92235,  7.92153,  8.95773, 23.569  , 14.0112 ,  6.20851,
         6.96098, 26.7469 , 56.8929 ],
       [10.097  , 10.2167 , 11.4239 ,

In [21]:
ytest[:10]

array([1, 2, 2, 2, 1, 2, 0, 0, 1, 0], dtype=int64)

In [23]:
clf.predict(xtest[:10,:])

array([1, 2, 2, 2, 1, 2, 0, 0, 1, 0], dtype=int64)

In [26]:
import pickle
s = pickle.dumps(clf)
f = open('E_nose_svm_model.txt', 'wb')
f.write(s)

22454633